In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
from config import *

from utils_functions import *

os.chdir(PATH_WORK_DIR)

%load_ext autoreload
%autoreload 2

In [ ]:
instruments_size = {k: 0 for k in INSTRUMENT_TO_IDX.keys()}
instrument_dirs = [f.path for f in os.scandir(PATH_TRAIN) if f.is_dir()]

for instrument_dir in instrument_dirs:
    instrument_name = Path(instrument_dir).stem
    num_of_files = len(os.listdir(instrument_dir))
    instruments_size[instrument_name] = num_of_files

instrument_frequency = sorted(instruments_size.items(), key=lambda x: x[1], reverse=True)

plt.figure(figsize=(15, 5), facecolor="w")
plt.bar(
    [INSTRUMENT_TO_FULLNAME[f[0]] for f in instrument_frequency],
    [f[1] for f in instrument_frequency],
)

In [ ]:
import torch
import torchaudio
import torchaudio.functional as F
import librosa

audio_path = "data/raw/train_sample/vio/001__[vio][nod][cou_fol]2194__1.wav"
audio, orig_sampling_rate = librosa.load(audio_path, sr=None)


In [ ]:
import torch
from torchmetrics.classification import F1Score
target = torch.tensor([0, 1, 0, 1, 0, 1])
preds = torch.tensor([0, 0, 1, 1, 0, 1])
f1 = F1Score(task="multiclass", num_classes=3, average=None)
f1(preds, target)